<a href="https://colab.research.google.com/github/bhaghighi/AG-CNN/blob/master/lstm_attention/Dataset_Seq_to_Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# got the data set from this link
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import drive
#drive.mount("/content/drive")
#os.chdir(r"/content/drive/My Drive/Colab Notebooks/Time-series-modeling")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''order of functin in the class does not matter. becasue when is running
all of functions are availabe in the instances. it is better to put
high level function first so that developer can understand the high level'''

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))


Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['il est petit et gros', 'he s short and fat']


In [ ]:
pairs[:100]

[['je vais bien', 'i m ok'],
 ['ca va', 'i m ok'],
 ['je suis gras', 'i m fat'],
 ['je suis gros', 'i m fat'],
 ['je suis en forme', 'i m fit'],
 ['je suis touche !', 'i m hit !'],
 ['je suis touchee !', 'i m hit !'],
 ['je suis malade', 'i m ill'],
 ['je suis triste', 'i m sad'],
 ['je suis timide', 'i m shy'],
 ['je suis mouille', 'i m wet'],
 ['je suis mouillee', 'i m wet'],
 ['il est mouille', 'he s wet'],
 ['je suis gras', 'i am fat'],
 ['je suis revenu', 'i m back'],
 ['me revoila', 'i m back'],
 ['je suis chauve', 'i m bald'],
 ['je suis occupe', 'i m busy'],
 ['je suis occupee', 'i m busy'],
 ['je suis calme', 'i m calm'],
 ['j ai froid', 'i m cold'],
 ['j en ai fini', 'i m done'],
 ['tout va bien', 'i m fine'],
 ['je vais bien', 'i m fine'],
 ['ca va', 'i m fine'],
 ['je suis libre !', 'i m free !'],
 ['je suis libre', 'i m free'],
 ['je suis disponible', 'i m free'],
 ['je suis repu !', 'i m full'],
 ['je suis rassasie !', 'i m full'],
 ['je suis content', 'i m glad'],
 ['je 